In [20]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from scipy.signal import butter, filtfilt
from scipy.signal import welch
from scipy.signal import correlate
from scipy.stats import pearsonr
import os

In [21]:
# システムパラメータ
takami_params = {
    'm1': 28.00,   # 質量1（脚）
    'm2': 53.00,   # 質量2（胴体）
    'L1': 0.90,    # 長さ1（脚）
    'L2': 0.88,    # 長さ2（胴体）
    'com1': 0.58,  # 質量中心1
    'com2': 0.32,  # 質量中心2
    'g': 9.81      # 重力加速度
}

# CoM計算関数
def calculate_com(q1, q2, params):
    q1_rad = np.deg2rad(q1)  # ラジアンに変換
    q2_rad = np.deg2rad(q2)

    m1 = params['m1']
    m2 = params['m2']
    L1 = params['L1']
    L2 = params['L2']
    r1 = params['com1']
    r2 = params['com2']

    # CoM位置計算
    X_CoM = (m1 * r1 * np.sin(q1_rad) + m2 * (L1 * np.sin(q1_rad) + r2 * np.sin(q1_rad + q2_rad))) / (m1 + m2)
    Z_CoM = (m1 * r1 * np.cos(q1_rad) + m2 * (L1 * np.cos(q1_rad) + r2 * np.cos(q1_rad + q2_rad))) / (m1 + m2)

    return pd.Series([X_CoM, Z_CoM], index=['com_x', 'com_z'])

# フォルダ内のすべてのCSVファイルを処理
def process_csv_files(folder_path):
    for file in os.listdir(folder_path):
        if file.endswith(".csv"):
            file_path = os.path.join(folder_path, file)
            print(f"Processing {file_path}")
            
            # CSV読み込み
            df = pd.read_csv(file_path)
            
            # COPの微分
            df['dCOP_s'] = df['COP_s'].diff().fillna(0)
            df['ddCOP_s'] = df['dCOP_s'].diff().fillna(0)

            # CoM計算
            df[['com_x_s', 'com_z_s']] = df.apply(lambda row: calculate_com(row['q1_s_true'], row['q2_s_true'], takami_params), axis=1)
            df[['dcom_x_s', 'dcom_z_s']] = df[['com_x_s', 'com_z_s']].diff().fillna(0)
            df[['ddcom_x_s', 'ddcom_z_s']] = df[['dcom_x_s', 'dcom_z_s']].diff().fillna(0)
            df[['dddcom_x_s', 'dddcom_z_s']] = df[['ddcom_x_s', 'ddcom_z_s']].diff().fillna(0)
            
            
            # 上書き保存
            df.to_csv(file_path, index=False)
            print(f"Saved: {file_path}")

# 実行（フォルダパスを指定）
folder_path = "new50withCOP"  # CSVファイルがあるフォルダのパスを設定
process_csv_files(folder_path)


Processing new50withCOP/EKF_Simulation_AllData_14.csv
Saved: new50withCOP/EKF_Simulation_AllData_14.csv
Processing new50withCOP/EKF_Simulation_AllData_28.csv
Saved: new50withCOP/EKF_Simulation_AllData_28.csv
Processing new50withCOP/EKF_Simulation_AllData_29.csv
Saved: new50withCOP/EKF_Simulation_AllData_29.csv
Processing new50withCOP/EKF_Simulation_AllData_01.csv
Saved: new50withCOP/EKF_Simulation_AllData_01.csv
Processing new50withCOP/EKF_Simulation_AllData_15.csv
Saved: new50withCOP/EKF_Simulation_AllData_15.csv
Processing new50withCOP/EKF_Simulation_AllData_03.csv
Saved: new50withCOP/EKF_Simulation_AllData_03.csv
Processing new50withCOP/EKF_Simulation_AllData_17.csv
Saved: new50withCOP/EKF_Simulation_AllData_17.csv
Processing new50withCOP/EKF_Simulation_AllData_16.csv
Saved: new50withCOP/EKF_Simulation_AllData_16.csv
Processing new50withCOP/EKF_Simulation_AllData_02.csv
Saved: new50withCOP/EKF_Simulation_AllData_02.csv
Processing new50withCOP/EKF_Simulation_AllData_06.csv
Saved: new

In [22]:
import pandas as pd
import numpy as np
import os
from scipy.stats import pearsonr, friedmanchisquare, wilcoxon

# システムパラメータ
takami_params = {
    'm1': 28.00,   # 質量1（脚）
    'm2': 53.00,   # 質量2（胴体）
    'L1': 0.90,    # 長さ1（脚）
    'L2': 0.88,    # 長さ2（胴体）
    'com1': 0.58,  # 質量中心1
    'com2': 0.32,  # 質量中心2
    'g': 9.81      # 重力加速度
}

# CoM計算関数
def calculate_com(q1, q2, params):
    q1_rad = np.deg2rad(q1)  # ラジアンに変換
    q2_rad = np.deg2rad(q2)

    m1 = params['m1']
    m2 = params['m2']
    L1 = params['L1']
    L2 = params['L2']
    r1 = params['com1']
    r2 = params['com2']

    # CoM位置計算
    X_CoM = (m1 * r1 * np.sin(q1_rad) + m2 * (L1 * np.sin(q1_rad) + r2 * np.sin(q1_rad + q2_rad))) / (m1 + m2)
    Z_CoM = (m1 * r1 * np.cos(q1_rad) + m2 * (L1 * np.cos(q1_rad) + r2 * np.cos(q1_rad + q2_rad))) / (m1 + m2)

    return pd.Series([X_CoM, Z_CoM], index=['com_x', 'com_z'])

# フォルダ内のすべてのCSVファイルを処理し、統合
def process_csv_files(folder_path):
    all_results = []
    for file in os.listdir(folder_path):
        if file.endswith(".csv"):
            file_path = os.path.join(folder_path, file)
            print(f"Processing {file_path}")

            # CSV読み込み
            df = pd.read_csv(file_path)
            
            # COPの微分
            df['dCOP_s'] = df['COP_s'].diff().fillna(0)
            df['ddCOP_s'] = df['dCOP_s'].diff().fillna(0)

            # CoM計算
            df[['com_x_s', 'com_z_s']] = df.apply(lambda row: calculate_com(row['q1_s_true'], row['q2_s_true'], takami_params), axis=1)
            df['dcom_x_s'] = df['com_x_s'].diff().fillna(0)
            df['ddcom_x_s'] = df['dcom_x_s'].diff().fillna(0)
            df['dddcom_x_s'] = df['ddcom_x_s'].diff().fillna(0)
            
            
            # 各変数のRMS計算
            rms_values = df[['COP_s', 'dCOP_s','ddCOP_s' ,'com_x_s', 'com_z_s', 'dcom_x_s','ddcom_x_s','dddcom_x_s']].apply(lambda x: np.sqrt(np.mean(x**2)))
            all_results.append(rms_values)
    
    # すべての結果を1つのデータフレームに統合
    final_df = pd.DataFrame(all_results)
    
    # Pearsonの相関計算
    correlation_matrix = final_df.corr(method='pearson')
    print("Correlation Matrix:")
    print(correlation_matrix)
    
    return final_df, correlation_matrix

# 実行（フォルダパスを指定）
folder_path = "new50withCOP"  # CSVファイルがあるフォルダのパスを設定
final_rms_df, correlation_matrix = process_csv_files(folder_path)

# 統合データを保存
final_rms_df.to_csv("rms_summary.csv", index=False)
print("RMS summary saved as rms_summary.csv")


Processing new50withCOP/EKF_Simulation_AllData_14.csv
Processing new50withCOP/EKF_Simulation_AllData_28.csv
Processing new50withCOP/EKF_Simulation_AllData_29.csv
Processing new50withCOP/EKF_Simulation_AllData_01.csv
Processing new50withCOP/EKF_Simulation_AllData_15.csv
Processing new50withCOP/EKF_Simulation_AllData_03.csv
Processing new50withCOP/EKF_Simulation_AllData_17.csv
Processing new50withCOP/EKF_Simulation_AllData_16.csv
Processing new50withCOP/EKF_Simulation_AllData_02.csv
Processing new50withCOP/EKF_Simulation_AllData_06.csv
Processing new50withCOP/EKF_Simulation_AllData_12.csv
Processing new50withCOP/EKF_Simulation_AllData_13.csv
Processing new50withCOP/EKF_Simulation_AllData_07.csv
Processing new50withCOP/EKF_Simulation_AllData_39.csv
Processing new50withCOP/EKF_Simulation_AllData_11.csv
Processing new50withCOP/EKF_Simulation_AllData_05.csv
Processing new50withCOP/EKF_Simulation_AllData_04.csv
Processing new50withCOP/EKF_Simulation_AllData_10.csv
Processing new50withCOP/EKF_

In [23]:
correlation_matrix

,COP_s,dCOP_s,ddCOP_s,com_x_s,com_z_s,dcom_x_s,ddcom_x_s,dddcom_x_s
COP_s,1.000000,0.688673,0.603634,0.363369,-0.028715,-0.227294,-0.363539,-0.354655
dCOP_s,0.688673,1.000000,0.967941,-0.058142,0.020420,-0.316726,-0.317968,-0.325119
ddCOP_s,0.603634,0.967941,1.000000,-0.068039,0.022015,-0.261218,-0.249809,-0.265837
com_x_s,0.363369,-0.058142,-0.068039,1.000000,-0.318653,0.068308,-0.187714,-0.172956
com_z_s,-0.028715,0.020420,0.022015,-0.318653,1.000000,-0.145800,-0.082341,-0.091290
dcom_x_s,-0.227294,-0.316726,-0.261218,0.068308,-0.145800,1.000000,0.779152,0.688785
ddcom_x_s,-0.363539,-0.317968,-0.249809,-0.187714,-0.082341,0.779152,1.000000,0.984639
dddcom_x_s,-0.354655,-0.325119,-0.265837,-0.172956,-0.091290,0.688785,0.984639,1.000000
